In [ ]:
# %%

import pandas as pd

from source.utils import FREQ_DIR, RESULT_DIR, UCS_DIR, confirm_dir
from source.utils.associate import (BINARY_ASSOC_ARGS, add_extra_am,
                                    associate_ucs, confirm_basic_ucs)
from source.utils.associate import convert_ucs_to_csv as ucs2csv
from source.utils.associate import get_associations_csv as init_am, AM_DF_DIR
from source.utils.associate import manipulate_ucs, seek_readable_ucs, adjust_assoc_columns
pd.set_option('display.float_format', '{:,.2f}'.format)

/home/arh234/anaconda3/envs/dev-sanpi/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# %% [markdown]

 set parameters:
 - `UNIT` options:
   - `'Bigr'`
   - `'Adv'`
   - `'Adj'`
 - `PAT_DIR` options:
   - any directory in `results/freq_out/` containing a `ucs_format/` subdir with a `Trig*.tsv` frequency table
   - developed with the following in mind:
     - `POSmirror`
     - `NEGmirror`
     - `ANYmirror` (which has been populated by simple concatenation in the shell)
 UNIT = 'Adv'

In [ ]:
UNIT = 'Bigr' 
# UNIT = 'Adv'
# PAT_DIR = 'POSmirror'
# PAT_DIR = 'NEGmirror'
PAT_DIR = 'POSmirror'
# FRQ_FLOOR = 3
FRQ_FLOOR = 10
# FRQ_FLOOR = 20
# FRQ_FLOOR = 50
# FRQ_FLOOR = 100
TRIG_TSV = FREQ_DIR.joinpath(
    f'{PAT_DIR}/ucs_format/Trig{UNIT}_frq-thrMIN-7.35f.tsv')
FOCUS = ['f', 'unexpected_f',
         'conservative_log_ratio',
         'am_p1_given2', 'am_p2_given1',
         'am_p1_given2_simple', 'am_p2_given1_simple',
         'am_log_likelihood',
        #  'mutual_information', 'am_odds_ratio_disc', 't_score',
         'N', 'f1', 'f2', 'E11',
         'l1', 'l2']

In [ ]:
# %%

TRIGGER_POLARITY = {
    'positive': {'all',
                 'always',
                 'both',
                 'either',
                 'every',
                 'everybody',
                 'everyone',
                 'everything',
                 'many',
                 'often',
                 'or',
                 'some',
                 'somebody',
                 'someone',
                 'something',
                 'somethings',
                 'sometimes'},
    'negative': {'barely',
                 'hardly',
                 'neither',
                 'never',
                 'no',
                 'nobody',
                 'none',
                 'nor',
                 'nothing',
                 'seldom',
                 'rarely',
                 'scarcely'},

}

In [ ]:
# %%

TRIGGER_QUANT = {
    'existential': {
        'some',
        'somebody',
        'someone',
        'something',
        'somethings',
        'either',
     'or',
        'sometimes'
    },
    'universal': {
        'all', 'every',
        'always',
        'everybody',
        'everyone',
        'both',
        'everything',
    },
    'not_exist': {
        'neither',
        'never',
        'no',
        'nobody',
        'none',
        'nor',
        'nothing',
    },
    'hedged_not_exist': {
        'barely',
        'hardly',
        'rarely',
        'scarcely',
        'seldom',
        'few'
    },
    'hedged_universal':{
     'many',
     'often',
     }
}

In [ ]:
# %%

def invert_set_dict(d: dict):
    return {v: k for k in d for v in d[k]}

# %% [markdown]

## 1. Run `seek_readable_ucs()` to generate consistent output path

In [ ]:
readable = seek_readable_ucs(min_freq=FRQ_FLOOR,
                             ucs_subdir='trigger_eval',
                             contained_counts_path=TRIG_TSV)
print(readable.relative_to(RESULT_DIR))

    > seeking `trigger_eval/POSmirror/readable/TrigBigr_frq-thrMIN-7.35f_min10x*` frequency data and initial associations...
ucs/trigger_eval/POSmirror/readable/TrigBigr_frq-thrMIN-7.35f_min10x.rsort-view_am-only.txt


# %% [markdown]

 Snippet of starting frequency data (`TRIG_TSV`)

In [ ]:
! head -5 {TRIG_TSV} | column -t

12233  something  as_simple
7372   something  completely_different
6933   all        too_familiar
6505   all        too_easy
5141   or         more_likely


# %% [markdown]

## 2. Run `confirm_basic_ucs()` (if needed)

In [ ]:
if not readable.is_file():
    basic_ucs_path = readable.parent.parent.joinpath(
        readable.name.replace('.rsort-view_am-only.txt', '.ds.gz'))
    print(f'Creating initial UCS table: `{basic_ucs_path.relative_to(RESULT_DIR)}')

    basic_ucs_path = confirm_basic_ucs(
        basic_ucs_path,
        freq_floor=FRQ_FLOOR,
        contained_counts_path=TRIG_TSV)

Creating initial UCS table: `ucs/trigger_eval/POSmirror/TrigBigr_frq-thrMIN-7.35f_min10x.ds.gz

## Creating initial UCS table...

```
( cat /share/compling/projects/sanpi/results/freq_out/POSmirror/ucs_format/TrigBigr_frq-thrMIN-7.35f.tsv \ 
  | ucs-make-tables --types --threshold=10 /share/compling/projects/sanpi/results/ucs/trigger_eval/POSmirror/TrigBigr_frq-thrMIN-7.35f_min10x.ds.gz ) \ 
  && ucs-sort /share/compling/projects/sanpi/results/ucs/trigger_eval/POSmirror/TrigBigr_frq-thrMIN-7.35f_min10x.ds.gz BY f2- f1- INTO /share/compling/projects/sanpi/results/ucs/trigger_eval/POSmirror/TrigBigr_frq-thrMIN-7.35f_min10x.ds.gz
== Note ==
    N = total number of tokens/all counts summed
    V = total number of rows/number of unique l1+l2 combinations before filtering to 10+ tokens
+ time to make table → 00:00:01.442
```

Saving initial frequency table in readable .txt format...

```
ucs-print -o /share/compling/projects/sanpi/results/ucs/trigger_eval/POSmirror/readable/TrigBigr_frq-thrM

# %% [markdown]

 Excerpt of initial UCS table

In [ ]:
init_readable = UCS_DIR.joinpath(
    f'trigger_eval/{PAT_DIR}/readable'
).joinpath(f'{TRIG_TSV.name.replace(".tsv","")}_min{FRQ_FLOOR}x.init.txt')
! head -7 {init_readable}

        l1  l2                                f     f2      f1        N  
----------  ----------------------------  -----  -----  ------  -------  
        or  as_simple                       791  14875  326614  1472077  
 something  as_simple                     12233  14875  318343  1472077  
       all  as_simple                        54  14875  249279  1472077  
      some  as_simple                       238  14875  114929  1472077  
     often  as_simple                       289  14875   92233  1472077  


# %% [markdown]

## 3. Run `associate_ucs()` (if needed)

In [ ]:
if not readable.is_file():
    associate_ucs(basic_ucs_path)

transform_ucs_log = f'/share/compling/projects/sanpi/logs/associate/ucs//ucs-{PAT_DIR}_Trig{UNIT}_frq-thrMIN-7-35f_min{FRQ_FLOOR}x*.log'
! head -15 `ls -t1 {transform_ucs_log} | head -1`
! echo '...'
! tail -2 `ls -t1 {transform_ucs_log} | head -1`


Calculating UCS associations...

```
bash /share/compling/projects/sanpi/script/transform_ucs.sh /share/compling/projects/sanpi/results/ucs/trigger_eval/POSmirror/TrigBigr_frq-thrMIN-7.35f_min10x.ds.gz
> log will be saved to: /share/compling/projects/sanpi/logs/associate/ucs//ucs-POSmirror_TrigBigr_frq-thrMIN-7-35f_min10x.2024-05-25_2212.log
...
```

+ time elapsed → 00:00:20.132
# Manipulating TrigBigr_frq-thrMIN-7-35f_min10x ucs table
path to this script: /share/compling/projects/sanpi/script/transform_ucs.sh
Sat May 25 22:12:27 EDT 2024
(TMP: /share/compling/projects/sanpi/results/ucs/trigger_eval/POSmirror/tmp/tmp_POSmirror-20240525-221227.TrigBigr_frq-thrMIN-7-35f_min10x)
## Initial Contingency Info

DATA SET FILE:  /share/compling/projects/sanpi/results/ucs/trigger_eval/POSmirror/TrigBigr_frq-thrMIN-7.35f_min10x.ds.gz

# Frequency signatures computed by the ucs-make-tables tool for relational cooccurrences.
# Sample size:  N = 1472077 tokens,  V = 330694 pair types.
# A frequenc

# %% [markdown]

### Define dictionary containing relevant vocab sizes
 !!! Warning This is a `#HACK`: \
     Rather than developing a command/code to retrieve the vocab sizes programmatically,
     I simply copied the values given in the log output of `transform_usc.sh`
     for each `PAT_DIR`+`UNIT` combination

 |      |   ANYmirror |   NEGmirror |   POSmirror |
 |:-----|------------:|------------:|------------:|
 | Bigr |     395,338 |      64,644 |     330,694 |
 | Adv  |      23,125 |       5,004 |      18,121 |
 | Adj  |      83,422 |      21,562 |      61,860 |

In [ ]:
VOCABS = {'ANYmirror': {'Adv': 23125, 'Bigr': 395338, 'Adj': 83422},
          'NEGmirror': {'Adv': 5004, 'Bigr': 64644, 'Adj': 21562},
          'POSmirror': {'Adv': 18121, 'Bigr': 330694, 'Adj': 61860}
          }  # HACK
VOCAB = VOCABS[PAT_DIR][UNIT]
pd.DataFrame(VOCABS)

,ANYmirror,NEGmirror,POSmirror
Adv,23125,5004,18121
Bigr,395338,64644,330694
Adj,83422,21562,61860


# %% [markdown]

## 4. Run `ucs_to_csv()` to convert `ucs/[PAT_DIR]/readable/*.txt` to format that `pandas` can parse as a dataframe

In [ ]:
! head -5 {readable}
csv_path = ucs2csv(readable)
print(f'CSV: `{csv_path.relative_to(RESULT_DIR)}`')

        l1  l2                                f           E11  am.log.likelihood  am.odds.ratio.disc    am.p1.given2     am.p2.given1  am.p1.given2.simple  am.p2.given1.simple      f1     f2        N  
----------  ----------------------------  -----  ------------  -----------------  ------------------  --------------  ---------------  -------------------  -------------------  ------  -----  -------  
 everybody  super_heady                      20     0.1082144        208.8248966        3.8782370732   0.99460279052    2.5109856e-03        1.00000000000        2.5109856e-03    7965     20  1472077  
everything  generally_real                   18     0.4831867        130.2460427        3.1277343963   0.97316819502    4.5551169e-04        1.00000000000        4.5551169e-04   39516     18  1472077  
everything  usually_accurate                 14     0.3758119        101.3010982        3.0218866915   0.97316555066    3.5428687e-04        1.00000000000        3.5428687e-04   39516     14  

# %% [markdown]

##

In [ ]:
trig_amdf = pd.read_csv(csv_path).convert_dtypes()
trig_amdf

,l1,l2,f,E11,am_log_likelihood,am_odds_ratio_disc,am_p1_given2,am_p2_given1,am_p1_given2_simple,am_p2_given1_simple,f1,f2,N
0,everybody,super_heady,20,0.11,208.82,3.88,0.99,0.00,1.00,0.00,7965,20,1472077
1,everything,generally_real,18,0.48,130.25,3.13,0.97,0.00,1.00,0.00,39516,18,1472077
2,everything,usually_accurate,14,0.38,101.30,3.02,0.97,0.00,1.00,0.00,39516,14,1472077
3,everything,generally_genuine,11,0.30,79.59,2.92,0.97,0.00,1.00,0.00,39516,11,1472077
4,everything,usually_correct,10,0.27,72.36,2.88,0.97,0.00,1.00,0.00,39516,10,1472077
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18684,or,very_excited,16,546.03,"-1,082.98",-1.63,-0.22,-0.00,0.01,0.00,326614,2461,1472077
18685,or,too_aware,13,502.10,"-1,008.91",-1.68,-0.22,-0.00,0.01,0.00,326614,2263,1472077
18686,or,too_common,19,"1,093.17","-2,275.13",-1.86,-0.22,-0.00,0.00,0.00,326614,4927,1472077
18687,or,very_special,18,"1,055.67","-2,199.94",-1.87,-0.22,-0.00,0.00,0.00,326614,4758,1472077


In [ ]:
# %%

trig_amdf['key'] = (trig_amdf.l1 + '~' +
                    trig_amdf.l2).astype('string')
trig_amdf = trig_amdf.set_index('key')
trig_amdf

,l1,l2,f,E11,am_log_likelihood,am_odds_ratio_disc,am_p1_given2,am_p2_given1,am_p1_given2_simple,am_p2_given1_simple,f1,f2,N
key,,,,,,,,,,,,,
everybody~super_heady,everybody,super_heady,20,0.11,208.82,3.88,0.99,0.00,1.00,0.00,7965,20,1472077
everything~generally_real,everything,generally_real,18,0.48,130.25,3.13,0.97,0.00,1.00,0.00,39516,18,1472077
everything~usually_accurate,everything,usually_accurate,14,0.38,101.30,3.02,0.97,0.00,1.00,0.00,39516,14,1472077
everything~generally_genuine,everything,generally_genuine,11,0.30,79.59,2.92,0.97,0.00,1.00,0.00,39516,11,1472077
everything~usually_correct,everything,usually_correct,10,0.27,72.36,2.88,0.97,0.00,1.00,0.00,39516,10,1472077
...,...,...,...,...,...,...,...,...,...,...,...,...,...
or~very_excited,or,very_excited,16,546.03,"-1,082.98",-1.63,-0.22,-0.00,0.01,0.00,326614,2461,1472077
or~too_aware,or,too_aware,13,502.10,"-1,008.91",-1.68,-0.22,-0.00,0.01,0.00,326614,2263,1472077
or~too_common,or,too_common,19,"1,093.17","-2,275.13",-1.86,-0.22,-0.00,0.00,0.00,326614,4927,1472077


# %% [markdown]

## 6. Save to `./results/assoc_df/`

In [ ]:
df_csv_path = AM_DF_DIR.joinpath(
    str(csv_path.relative_to(UCS_DIR))
    .replace('/readable', '')
    .replace('.rsort-view_am-only', ''))

if not df_csv_path.is_file():
    confirm_dir(df_csv_path.parent)
    trig_amdf.to_csv(df_csv_path)

df_pkl_path = df_csv_path.with_suffix('.pkl.gz')
if not df_pkl_path.is_file():
    trig_amdf.to_pickle(df_csv_path.with_suffix('.pkl.gz'))

# %% [markdown]

## 7. Add additional AM via `add_extra_am()`

In [ ]:
ex_trig_amdf = add_extra_am(df=trig_amdf,
                            verbose=True,
                            vocab=VOCAB,
                            metrics=['t_score', 'mutual_information']
                            ).convert_dtypes()


Preview of Extended Measures (rounded)

| key                            |   t_score |   mutual_information |   deltaP_min |   deltaP_max |   deltaP_max_abs |   deltaP_product |   unexpected_f |   unexpected_ratio |
|:-------------------------------|----------:|---------------------:|-------------:|-------------:|-----------------:|-----------------:|---------------:|-------------------:|
| everybody~super_heady          |      4.45 |                 2.27 |         0.00 |         0.99 |             0.99 |             0.00 |          19.89 |               0.99 |
| everything~generally_real      |      4.13 |                 1.57 |         0.00 |         0.97 |             0.97 |             0.00 |          17.52 |               0.97 |
| everything~usually_accurate    |      3.64 |                 1.57 |         0.00 |         0.97 |             0.97 |             0.00 |          13.62 |               0.97 |
| everything~generally_genuine   |      3.23 |                 1.57 |         0

#%% [markdown]

 Add trigger features as columns: polarity and quantification type

In [ ]:
def add_feature(triggers:pd.Series, 
                group_dict: dict) -> pd.Series:
    assign_dict = invert_set_dict(group_dict)
    return triggers.apply(lambda x: assign_dict[x] if x in assign_dict.keys() else '')

ex_trig_amdf['polarity'] = add_feature(ex_trig_amdf.l1, TRIGGER_POLARITY)
ex_trig_amdf['quant'] = add_feature(ex_trig_amdf.l1, TRIGGER_QUANT)

# %% [markdown]

 Save extended AM tables to `extra/` subdirectory if not already saved

In [ ]:
df_extra_csv = df_csv_path.parent / 'extra' / \
    df_csv_path.name.replace('.csv', '_extra.csv')
print(df_extra_csv)
if not df_extra_csv.is_file():
    confirm_dir(df_extra_csv.parent)
    ex_trig_amdf.to_csv(df_extra_csv)

df_extra_pkl = df_extra_csv.with_suffix('.pkl.gz')
if not df_extra_pkl.is_file():
    ex_trig_amdf.to_pickle(df_extra_pkl)

/share/compling/projects/sanpi/results/assoc_df/trigger_eval/POSmirror/extra/TrigBigr_frq-thrMIN-7.35f_min10x_extra.csv


In [ ]:
# %%

ex_trig_full = ex_trig_amdf.copy()
ex_trig_abbr = adjust_assoc_columns(
    ex_trig_amdf[[c for c in ['polarity', 'quant'] + FOCUS if c in ex_trig_amdf.columns]]).sort_values('LRC', ascending=False)
cols = ex_trig_abbr.columns

# %% [markdown]

 Define lexical items with given lean shown in binary environment evaluation

In [ ]:
pos_prone = {
    'Adj': [
        'unrelated',
        'unable',
        'akin',
        'larger',
        'different',
        'familiar',
        'similar',
        'likely', 
        'brief', 
        'unaware'
    ],
    'Adv': [
        'slightly', 
        'definitely',
        'utterly',
        # LRC top
        'pretty', 
        'rather', 
        'plain', 
        'fairly', 
        'somewhat', 
        'otherwise', 
        'downright',
        'relatively', 
        # G2 top
        # 'very', 
        # 'even', 
        # 'just', 
        # dP1 top (and odds ratio disc)
        'plain', 
        'maybe'
    ], 
    'Bigr': [
        # G2 top
        'completely_different', 
        'too_familiar',
        'even_better',
        # dP1 top
        'quite_different', 
        'too_real', 
        'well_aware', 
        # LRC top
        'too_common', 
        'entirely_different'
        ]}
neg_prone = {
    'Bigr': [
        # LRC top
        'quite_sure',
        'really_sure', 
        'too_early', 
        'too_pleased', 
        'too_fancy',
        # dP1 top
        'entirely_sure', 
        'ever_easy', 
        'ever_perfect', 
        'particularly_surprising',
        'particularly_new',
        # G2 top
        'too_late', 
        'more_important',
        'so_easy', 
        'as_good', 
        'too_old'
        ], 
    'Adv': [
        'yet',
        #LRC top
        'ever',
        'any', 
        'longer', 
        'necessarily',
        'that',
        #dP1 top
        'before', 
        'wise',  #? How is this used as an adverb?
        'earthly',
        'remotely',
        'exactly',
        # G2 top
        'particularly', 
        'too',
        # 'inherently'
        ],
    'Adj': [
        # LRC top
        'early', 
        'late', 
        'fancy', 
        'alone', 
        'sure',
        # dP1 top
        'shabby',
        'demoralizing',
        'alone',
        'aggravating',
        'groundbreaking',
        'eventful',
        #G2 top
        'important', 
        'frustrating',
        'evident', 
        'certain'
        ]
    }
def sort_prone_by_f2(prone_list, amdf):
    return amdf.copy().loc[amdf.l2.isin(prone_list), ['f2','l2']].drop_duplicates().reset_index(drop=True).set_index('l2').round(1).sort_values(['f2'], ascending=False).index.to_list()
pos_prone[UNIT] = sort_prone_by_f2(pos_prone[UNIT], ex_trig_abbr)
neg_prone[UNIT] = sort_prone_by_f2(neg_prone[UNIT], ex_trig_abbr)
pos_prone[UNIT]

['completely_different',
 'too_familiar',
 'even_better',
 'too_common',
 'entirely_different',
 'quite_different',
 'too_real',
 'well_aware']

# %% [markdown]

## Strongest associations for each polarity by metric

In [ ]:
def show_metric_top(amdf: pd.DataFrame,
                    metric: str,
                    k=5, cols=[None]):
    if not any(cols):
        cols = amdf.columns
    return (pd.concat((polar_df.nlargest(k, metric)
                       for pol, polar_df in amdf.groupby('polarity')))
            .loc[:, [metric] + cols[cols != metric].to_list()]
            .reset_index(drop=True).set_index(['polarity', 'l1', 'l2'])
            )
def update_prone(exdf, pos_prone, neg_prone) -> dict: 
    prone_dict = {'positive':pos_prone, 
                  'negative': neg_prone}
    for polarity, prone in prone_dict.items():
        try: 
            polar_ex = exdf.loc[polarity, :]
        except KeyError: 
            continue
        else: 
                
            for topw in polar_ex.reset_index().l2.head(2).squeeze(): 
                if topw not in prone[UNIT]: 
                    prone[UNIT].append(topw)
            prone_dict[polarity] = prone
    return prone_dict['positive'], prone_dict['negative']

# %% [markdown]

 Top consevative log ratio $LRC$ values

In [ ]:
exdf = show_metric_top(ex_trig_abbr,"LRC")
pos_prone, neg_prone = update_prone(exdf, pos_prone, neg_prone)
exdf

LRC        quant     f  unexp_f  dP1  \
polarity l1         l2                                                      
positive all        too_common      7.83    universal  4882 4,047.67 0.82   
                    too_familiar    7.65    universal  6933 5,743.06 0.82   
         everything all_right       7.32    universal  2166 2,098.33 0.83   
         all        too_aware       6.92    universal  2229 1,845.79 0.82   
         something  seriously_wrong 6.77  existential  1964 1,536.03 0.78   

                                     dP2  dP1_simple  dP2_simple        G2  \
polarity l1         l2                                                       
positive all        too_common      0.02        0.99        0.02 16,923.69   
                    too_familiar    0.03        0.99        0.03 23,822.15   
         everything all_right       0.05        0.86        0.05 13,758.73   
         all        too_aware       0.01        0.98        0.01  7,592.94   
         something  seriously_wrong 0.01        0.99        0.01  5,855.38   

                                           N      f1    f2    exp_f  
polarity l1         l2                                               
positive all        too_common       1472077  249279  4927   834.33  
                    too_familiar     1472077  249279  7027 1,189.94  
         everything all_right        1472077   39516  2521    67.67  
         all        too_aware        1472077  249279  2263   383.21  
         something  seriously_wrong  1472077  318343  1979   427.97

# %% [markdown]

 Top $\Delta P(\texttt{trigger}|\texttt{adv})$ values

In [ ]:
exdf = show_metric_top(ex_trig_abbr, 'dP1')
pos_prone, neg_prone = update_prone(exdf, pos_prone, neg_prone)
exdf

dP1      quant   f  unexp_f  LRC  dP2  \
polarity l1         l2                                                         
positive everybody  super_heady       0.99  universal  20    19.89 3.48 0.00   
         everything generally_real    0.97  universal  18    17.52 0.96 0.00   
                    usually_accurate  0.97  universal  14    13.62 0.53 0.00   
                    generally_genuine 0.97  universal  11    10.70 0.11 0.00   
                    usually_correct   0.97  universal  10     9.73 0.00 0.00   

                                       dP1_simple  dP2_simple     G2        N  \
polarity l1         l2                                                          
positive everybody  super_heady              1.00        0.00 208.82  1472077   
         everything generally_real           1.00        0.00 130.25  1472077   
                    usually_accurate         1.00        0.00 101.30  1472077   
                    generally_genuine        1.00        0.00  79.59  1472077   
                    usually_correct          1.00        0.00  72.36  1472077   

                                          f1  f2  exp_f  
polarity l1         l2                                   
positive everybody  super_heady         7965  20   0.11  
         everything generally_real     39516  18   0.48  
                    usually_accurate   39516  14   0.38  
                    generally_genuine  39516  11   0.30  
                    usually_correct    39516  10   0.27

# %% [markdown]

 Top conditional probability $P(\texttt{trigger}|\texttt{adv})$ values

In [ ]:
exdf = show_metric_top(ex_trig_abbr, 'dP1_simple')
pos_prone, neg_prone = update_prone(exdf, pos_prone, neg_prone)
exdf

dP1_simple             quant    f  \
polarity l1        l2                                                           
positive everybody super_heady                    1.00         universal   20   
         many      instantly_bookable             1.00  hedged_universal   31   
         or        otherwise_inappropriate        1.00       existential  199   
                   otherwise_objectionable        1.00       existential  193   
         all       too_commonplace                1.00         universal  124   

                                            unexp_f  LRC  dP1  dP2  \
polarity l1        l2                                                
positive everybody super_heady                19.89 3.48 0.99 0.00   
         many      instantly_bookable         30.01 1.58 0.97 0.00   
         or        otherwise_inappropriate   154.85 1.26 0.78 0.00   
                   otherwise_objectionable   150.18 1.22 0.78 0.00   
         all       too_commonplace           103.00 1.05 0.83 0.00   

                                            dP2_simple     G2        N  \
polarity l1        l2                                                    
positive everybody super_heady                    0.00 208.82  1472077   
         many      instantly_bookable             0.00 213.39  1472077   
         or        otherwise_inappropriate        0.00 599.34  1472077   
                   otherwise_objectionable        0.00 581.27  1472077   
         all       too_commonplace                0.00 440.46  1472077   

                                                f1   f2  exp_f  
polarity l1        l2                                           
positive everybody super_heady                7965   20   0.11  
         many      instantly_bookable        47133   31   0.99  
         or        otherwise_inappropriate  326614  199  44.15  
                   otherwise_objectionable  326614  193  42.82  
         all       too_commonplace          249279  124  21.00

# %% [markdown]

 Top $\Delta P(\texttt{adv}|\texttt{trigger})$ values

In [ ]:
exdf = show_metric_top(ex_trig_abbr, 'dP2')
pos_prone, neg_prone = update_prone(exdf, pos_prone, neg_prone)
exdf

dP2        quant      f  unexp_f  LRC  dP1  \
polarity l1         l2                                                         
positive somethings more_likely  0.07  existential     12    11.17 1.58 0.00   
         everything all_right    0.05    universal   2166 2,098.33 7.32 0.83   
         something  as_simple    0.04  existential  12233 9,016.22 3.89 0.61   
         all        too_familiar 0.03    universal   6933 5,743.06 7.65 0.82   
         someone    so_young     0.03  existential    811   786.52 6.11 0.65   

                                  dP1_simple  dP2_simple        G2        N  \
polarity l1         l2                                                        
positive somethings more_likely         0.00        0.08     42.50  1472077   
         everything all_right           0.86        0.05 13,758.73  1472077   
         something  as_simple           0.82        0.04 25,168.08  1472077   
         all        too_familiar        0.99        0.03 23,822.15  1472077   
         someone    so_young            0.67        0.03  4,825.80  1472077   

                                      f1     f2    exp_f  
polarity l1         l2                                    
positive somethings more_likely      157   7819     0.83  
         everything all_right      39516   2521    67.67  
         something  as_simple     318343  14875 3,216.78  
         all        too_familiar  249279   7027 1,189.94  
         someone    so_young       29662   1215    24.48

# %% [markdown]

 Top conditional probability $P(\texttt{adv}|\texttt{trigger})$ values

In [ ]:
exdf = show_metric_top(ex_trig_abbr, 'dP2_simple')
pos_prone, neg_prone = update_prone(exdf, pos_prone, neg_prone)
exdf

dP2_simple        quant      f  unexp_f  \
polarity l1         l2                                                      
positive somethings more_likely         0.08  existential     12    11.17   
         everything all_right           0.05    universal   2166 2,098.33   
         something  as_simple           0.04  existential  12233 9,016.22   
         all        too_familiar        0.03    universal   6933 5,743.06   
         someone    so_young            0.03  existential    811   786.52   

                                  LRC  dP1  dP2  dP1_simple        G2  \
polarity l1         l2                                                  
positive somethings more_likely  1.58 0.00 0.07        0.00     42.50   
         everything all_right    7.32 0.83 0.05        0.86 13,758.73   
         something  as_simple    3.89 0.61 0.04        0.82 25,168.08   
         all        too_familiar 7.65 0.82 0.03        0.99 23,822.15   
         someone    so_young     6.11 0.65 0.03        0.67  4,825.80   

                                        N      f1     f2    exp_f  
polarity l1         l2                                             
positive somethings more_likely   1472077     157   7819     0.83  
         everything all_right     1472077   39516   2521    67.67  
         something  as_simple     1472077  318343  14875 3,216.78  
         all        too_familiar  1472077  249279   7027 1,189.94  
         someone    so_young      1472077   29662   1215    24.48

# %% [markdown]

 Top log-likelihood $G^2$ values

In [ ]:
exdf = show_metric_top(ex_trig_abbr, 'G2')
pos_prone, neg_prone = update_prone(exdf, pos_prone, neg_prone)
exdf

G2        quant      f  \
polarity l1        l2                                                   
positive something as_simple            25,168.08  existential  12233   
         all       too_familiar         23,822.15    universal   6933   
                   too_easy             18,837.78    universal   6505   
         something completely_different 17,236.33  existential   7372   
         all       too_common           16,923.69    universal   4882   

                                         unexp_f  LRC  dP1  dP2  dP1_simple  \
polarity l1        l2                                                         
positive something as_simple            9,016.22 3.89 0.61 0.04        0.82   
         all       too_familiar         5,743.06 7.65 0.82 0.03        0.99   
                   too_easy             5,282.21 5.16 0.74 0.03        0.90   
         something completely_different 5,571.25 4.53 0.67 0.02        0.89   
         all       too_common           4,047.67 7.83 0.82 0.02        0.99   

                                         dP2_simple        N      f1     f2  \
polarity l1        l2                                                         
positive something as_simple                   0.04  1472077  318343  14875   
         all       too_familiar                0.03  1472077  249279   7027   
                   too_easy                    0.03  1472077  249279   7221   
         something completely_different        0.02  1472077  318343   8327   
         all       too_common                  0.02  1472077  249279   4927   

                                           exp_f  
polarity l1        l2                             
positive something as_simple            3,216.78  
         all       too_familiar         1,189.94  
                   too_easy             1,222.79  
         something completely_different 1,800.75  
         all       too_common             834.33

In [ ]:
# %%

sig_trig_abbr = ex_trig_abbr.loc[ex_trig_abbr.LRC.abs() > 1, :]
def show_prone_ex(amdf, prone_list, list_index:int=0):
    try: 
        example = prone_list[list_index]

    except IndexError: 
        return list_index + 1, f'No {list_index}th entry'
    else:
        print(f'>> {example} <<')
        return list_index+ 1, amdf.filter(regex=f'~{example}$', axis=0).iloc[:10, :]

# %% [markdown]

 ---
 Significant Examples for Items demonstrated positive polarity environment lean overall (Top 10 by $|LRC|>1$)

In [ ]:
prone = pos_prone[UNIT]
prone

['completely_different',
 'too_familiar',
 'even_better',
 'too_common',
 'entirely_different',
 'quite_different',
 'too_real',
 'well_aware',
 'super_heady',
 'generally_real',
 'instantly_bookable',
 'more_likely',
 'all_right',
 'as_simple']

In [ ]:
# %%

ix = 0
ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> completely_different <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
something~completely_different,positive,existential,7372,"5,571.25",4.53,0.67,0.02,0.89,0.02,"17,236.33",1472077,318343,8327,"1,800.75",something,completely_different
both~completely_different,positive,universal,42,-240.87,-1.53,-0.03,-0.00,0.01,0.00,-329.88,1472077,50007,8327,282.87,both,completely_different
everyone~completely_different,positive,universal,11,-171.86,-1.62,-0.02,-0.01,0.00,0.00,-286.42,1472077,32327,8327,182.86,everyone,completely_different
often~completely_different,positive,hedged_universal,57,-464.73,-2.19,-0.06,-0.01,0.01,0.00,-706.69,1472077,92233,8327,521.73,often,completely_different
many~completely_different,positive,hedged_universal,10,-256.61,-2.20,-0.03,-0.01,0.00,0.00,-457.10,1472077,47133,8327,266.61,many,completely_different
sometimes~completely_different,positive,existential,38,-384.59,-2.22,-0.05,-0.01,0.00,0.00,-606.62,1472077,74707,8327,422.59,sometimes,completely_different
all~completely_different,positive,universal,207,"-1,203.08",-2.42,-0.15,-0.01,0.02,0.00,"-1,816.93",1472077,249279,8327,"1,410.08",all,completely_different
or~completely_different,positive,existential,214,"-1,633.54",-2.87,-0.20,-0.01,0.03,0.00,"-2,736.03",1472077,326614,8327,"1,847.54",or,completely_different
always~completely_different,positive,universal,14,-450.98,-2.95,-0.05,-0.01,0.00,0.00,-831.89,1472077,82200,8327,464.98,always,completely_different


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> too_familiar <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
all~too_familiar,positive,universal,6933,"5,743.06",7.65,0.82,0.03,0.99,0.03,"23,822.15",1472077,249279,7027,"1,189.94",all,too_familiar
many~too_familiar,positive,hedged_universal,15,-209.99,-1.84,-0.03,-0.00,0.00,0.00,-346.13,1472077,47133,7027,224.99,many,too_familiar
some~too_familiar,positive,existential,11,-537.62,-3.29,-0.08,-0.01,0.00,0.00,"-1,035.39",1472077,114929,7027,548.62,some,too_familiar
something~too_familiar,positive,existential,18,"-1,501.62",-4.82,-0.21,-0.01,0.00,0.00,"-3,229.20",1472077,318343,7027,"1,519.62",something,too_familiar
or~too_familiar,positive,existential,16,"-1,543.10",-4.92,-0.22,-0.01,0.00,0.00,"-3,348.54",1472077,326614,7027,"1,559.10",or,too_familiar


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> even_better <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
or~even_better,positive,existential,4352,"3,000.57",2.90,0.49,0.01,0.71,0.01,"6,727.58",1472077,326614,6091,"1,351.43",or,even_better
both~even_better,positive,universal,25,-181.91,-1.46,-0.03,-0.00,0.00,0.00,-264.41,1472077,50007,6091,206.91,both,even_better
some~even_better,positive,existential,94,-381.54,-1.59,-0.06,-0.00,0.02,0.00,-485.04,1472077,114929,6091,475.54,some,even_better
many~even_better,positive,hedged_universal,14,-181.02,-1.66,-0.03,-0.00,0.00,0.00,-294.51,1472077,47133,6091,195.02,many,even_better
often~even_better,positive,hedged_universal,54,-327.63,-1.79,-0.05,-0.00,0.01,0.00,-463.77,1472077,92233,6091,381.63,often,even_better
always~even_better,positive,universal,10,-330.12,-2.58,-0.05,-0.00,0.00,0.00,-609.71,1472077,82200,6091,340.12,always,even_better
all~even_better,positive,universal,59,-972.44,-3.31,-0.16,-0.00,0.01,0.00,"-1,787.80",1472077,249279,6091,"1,031.44",all,even_better


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> too_common <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
all~too_common,positive,universal,4882,"4,047.67",7.83,0.82,0.02,0.99,0.02,"16,923.69",1472077,249279,4927,834.33,all,too_common
or~too_common,positive,existential,19,"-1,074.17",-4.33,-0.22,-0.00,0.00,0.00,"-2,275.13",1472077,326614,4927,"1,093.17",or,too_common
something~too_common,positive,existential,11,"-1,054.48",-4.48,-0.21,-0.00,0.00,0.00,"-2,277.65",1472077,318343,4927,"1,065.48",something,too_common


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> entirely_different <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
something~entirely_different,positive,existential,3115,"2,401.36",5.31,0.73,0.01,0.94,0.01,"8,227.85",1472077,318343,3300,713.64,something,entirely_different
often~entirely_different,positive,hedged_universal,15,-191.76,-1.76,-0.06,-0.00,0.00,0.00,-316.89,1472077,92233,3300,206.76,often,entirely_different
or~entirely_different,positive,existential,45,-687.18,-3.14,-0.21,-0.00,0.01,0.00,"-1,294.57",1472077,326614,3300,732.18,or,entirely_different
all~entirely_different,positive,universal,14,-544.82,-3.39,-0.17,-0.00,0.00,0.00,"-1,089.59",1472077,249279,3300,558.82,all,entirely_different


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> quite_different <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
something~quite_different,positive,existential,2268,"1,585.50",2.89,0.50,0.01,0.72,0.01,"3,637.99",1472077,318343,3156,682.50,something,quite_different
always~quite_different,positive,universal,18,-158.23,-1.44,-0.05,-0.00,0.01,0.00,-242.91,1472077,82200,3156,176.23,always,quite_different
or~quite_different,positive,existential,34,-666.23,-3.30,-0.21,-0.00,0.01,0.00,"-1,294.82",1472077,326614,3156,700.23,or,quite_different


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> too_real <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
all~too_real,positive,universal,2250,"1,855.78",6.20,0.80,0.01,0.97,0.01,"7,353.85",1472077,249279,2328,394.22,all,too_real
or~too_real,positive,existential,22,-494.52,-3.15,-0.21,-0.00,0.01,0.00,-975.30,1472077,326614,2328,516.52,or,too_real


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> well_aware <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
everyone~well_aware,positive,universal,391,354.28,3.30,0.21,0.01,0.23,0.01,"1,228.20",1472077,32327,1672,36.72,everyone,well_aware
everybody~well_aware,positive,universal,65,55.95,1.87,0.03,0.01,0.04,0.01,146.75,1472077,7965,1672,9.05,everybody,well_aware
all~well_aware,positive,universal,821,537.87,1.84,0.32,0.00,0.49,0.00,915.79,1472077,249279,1672,283.13,all,well_aware
or~well_aware,positive,existential,34,-336.97,-2.37,-0.20,-0.00,0.02,0.00,-592.47,1472077,326614,1672,370.97,or,well_aware
something~well_aware,positive,existential,24,-337.58,-2.57,-0.20,-0.00,0.01,0.00,-625.75,1472077,318343,1672,361.58,something,well_aware


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> super_heady <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
everybody~super_heady,positive,universal,20,19.89,3.48,0.99,0.00,1.00,0.00,208.82,1472077,7965,20,0.11,everybody,super_heady


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> generally_real <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,


# %% [markdown]

 ---
## Examples for Items demonstrating *Negative* polarity environment lean overall (Top 10 by $|LRC|>1$)

In [ ]:
prone_list = neg_prone[UNIT]
prone = sig_trig_abbr.copy().loc[sig_trig_abbr.l2.isin(prone_list), ['f2','l2']].drop_duplicates().reset_index(drop=True).set_index('l2').round(1).sort_values(['f2'], ascending=False).index.to_list()
prone

['more_important', 'as_good', 'too_late', 'so_easy', 'too_old', 'too_pleased']

In [ ]:
# %%

ix = 0
ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> more_important <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
all~more_important,positive,universal,3987,"2,124.96",1.32,0.19,0.01,0.36,0.02,"2,381.12",1472077,249279,10996,"1,862.04",all,more_important
some~more_important,positive,existential,321,-537.49,-1.03,-0.05,-0.01,0.03,0.00,-474.17,1472077,114929,10996,858.49,some,more_important
or~more_important,positive,existential,1067,"-1,372.71",-1.15,-0.13,-0.01,0.10,0.00,"-1,197.32",1472077,326614,10996,"2,439.71",or,more_important
everything~more_important,positive,universal,61,-234.17,-1.26,-0.02,-0.01,0.01,0.00,-282.51,1472077,39516,10996,295.17,everything,more_important
both~more_important,positive,universal,44,-329.54,-1.90,-0.03,-0.01,0.00,0.00,-483.24,1472077,50007,10996,373.54,both,more_important
everyone~more_important,positive,universal,15,-226.47,-1.93,-0.02,-0.01,0.00,0.00,-375.95,1472077,32327,10996,241.47,everyone,more_important
many~more_important,positive,hedged_universal,22,-330.07,-2.30,-0.03,-0.01,0.00,0.00,-550.67,1472077,47133,10996,352.07,many,more_important


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> as_good <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
someone~as_good,positive,existential,189,139.96,1.42,0.06,0.00,0.08,0.01,239.07,1472077,29662,2434,49.04,someone,as_good
somebody~as_good,positive,existential,29,24.57,1.21,0.01,0.01,0.01,0.01,60.26,1472077,2682,2434,4.43,somebody,as_good


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> too_late <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
often~too_late,positive,hedged_universal,650,529.58,2.54,0.28,0.01,0.34,0.01,"1,309.51",1472077,92233,1922,120.42,often,too_late
some~too_late,positive,existential,16,-134.06,-1.28,-0.07,-0.00,0.01,0.00,-206.55,1472077,114929,1922,150.06,some,too_late


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> so_easy <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
everything~so_easy,positive,universal,207,178.63,2.51,0.17,0.00,0.20,0.01,499.29,1472077,39516,1057,28.37,everything,so_easy
or~so_easy,positive,existential,42,-192.52,-1.50,-0.18,-0.00,0.04,0.00,-282.63,1472077,326614,1057,234.52,or,so_easy


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> too_old <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
either~too_old,positive,existential,40,37.46,2.72,0.04,0.01,0.04,0.01,147.32,1472077,3886,964,2.54,either,too_old
or~too_old,positive,existential,521,307.11,1.52,0.32,0.00,0.54,0.00,461.46,1472077,326614,964,213.89,or,too_old


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> too_pleased <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
all~too_pleased,positive,universal,55,43.48,1.86,0.64,0.00,0.81,0.00,133.82,1472077,249279,68,11.52,all,too_pleased


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, [r'exactly\w*'] if UNIT!='Adj' else [r'un\w+'])
exdf

>> exactly\w* <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
everything~exactly_right,positive,universal,35,32.77,2.91,0.39,0.00,0.42,0.00,142.86,1472077,39516,83,2.23,everything,exactly_right


In [ ]:
show_metric_top(sig_trig_abbr.filter(like='too_', axis=0), 'dP1', 15)

dP1      quant     f  unexp_f  LRC  dP2  \
polarity l1  l2                                                         
positive all too_commonplace 0.83  universal   124   103.00 1.05 0.00   
             too_evident     0.83  universal   408   338.74 2.79 0.00   
             too_common      0.82  universal  4882 4,047.67 7.83 0.02   
             too_possible    0.82  universal   144   119.45 1.27 0.00   
             too_familiar    0.82  universal  6933 5,743.06 7.65 0.03   
             too_typical     0.82  universal   284   235.40 4.12 0.00   
             too_clear       0.82  universal  1095   907.20 6.39 0.00   
             too_aware       0.82  universal  2229 1,845.79 6.92 0.01   
             too_apparent    0.82  universal   556   460.32 5.50 0.00   
             too_prevalent   0.81  universal   211   174.42 4.00 0.00   
             too_likely      0.81  universal   282   233.06 4.50 0.00   
             too_human       0.80  universal   628   518.78 5.49 0.00   
             too_true        0.80  universal   471   388.70 5.13 0.00   
             too_real        0.80  universal  2250 1,855.78 6.20 0.01   
             too_relevant    0.80  universal    82    67.61 2.27 0.00   

                              dP1_simple  dP2_simple        G2        N  \
polarity l1  l2                                                           
positive all too_commonplace        1.00        0.00    440.46  1472077   
             too_evident            1.00        0.00  1,436.00  1472077   
             too_common             0.99        0.02 16,923.69  1472077   
             too_possible           0.99        0.00    499.94  1472077   
             too_familiar           0.99        0.03 23,822.15  1472077   
             too_typical            0.99        0.00    976.73  1472077   
             too_clear              0.99        0.00  3,748.06  1472077   
             too_aware              0.98        0.01  7,592.94  1472077   
             too_apparent           0.98        0.00  1,886.75  1472077   
             too_prevalent          0.98        0.00    703.87  1472077   
             too_likely             0.98        0.00    938.53  1472077   
             too_human              0.97        0.00  2,080.91  1472077   
             too_true               0.97        0.00  1,545.28  1472077   
             too_real               0.97        0.01  7,353.85  1472077   
             too_relevant           0.96        0.00    266.42  1472077   

                                  f1    f2    exp_f  
polarity l1  l2                                      
positive all too_commonplace  249279   124    21.00  
             too_evident      249279   409    69.26  
             too_common       249279  4927   834.33  
             too_possible     249279   145    24.55  
             too_familiar     249279  7027 1,189.94  
             too_typical      249279   287    48.60  
             too_clear        249279  1109   187.80  
             too_aware        249279  2263   383.21  
             too_apparent     249279   565    95.68  
             too_prevalent    249279   216    36.58  
             too_likely       249279   289    48.94  
             too_human        249279   645   109.22  
             too_true         249279   486    82.30  
             too_real         249279  2328   394.22  
             too_relevant     249279    85    14.39

In [ ]:
show_metric_top(sig_trig_abbr.filter(like='terribly_', axis=0), 'dP1', 15)

dP1        quant     f  unexp_f   LRC  \
polarity l1        l2                                                       
positive something terribly_wrong  0.75  existential  1571 1,219.59  5.59   
         all       terribly_wrong -0.16    universal    20  -255.17 -2.19   
         or        terribly_wrong -0.21  existential    17  -343.54 -2.76   

                                    dP2  dP1_simple  dP2_simple       G2  \
polarity l1        l2                                                      
positive something terribly_wrong  0.00        0.97        0.00 4,369.79   
         all       terribly_wrong -0.00        0.01        0.00  -451.25   
         or        terribly_wrong -0.00        0.01        0.00  -669.58   

                                         N      f1    f2  exp_f  
polarity l1        l2                                            
positive something terribly_wrong  1472077  318343  1625 351.41  
         all       terribly_wrong  1472077  249279  1625 275.17  
         or        terribly_wrong  1472077  326614  1625 360.54

In [ ]:
show_metric_top(sig_trig_abbr.filter(like='quite_', axis=0), 'dP1', 15)

dP1             quant     f  unexp_f  \
polarity l1        l2                                                          
positive all       quite_innovative    0.77         universal   370   303.62   
         something quite_special       0.72       existential   404   310.79   
                   quite_magical       0.69       existential    95    72.29   
                   quite_extraordinary 0.62       existential   164   121.40   
         some      quite_graphic       0.58       existential    31    27.33   
         something quite_profound      0.58       existential    61    44.35   
         some      quite_sophisticated 0.58       existential    32    28.17   
                   quite_elaborate     0.53       existential    45    39.22   
         something quite_unique        0.52       existential   224   158.47   
         some      quite_obscure       0.52       existential    27    23.49   
         something quite_remarkable    0.51       existential   146   102.75   
                   quite_different     0.50       existential  2268 1,585.50   
         often     quite_poor          0.49  hedged_universal    35    31.05   
         some      quite_recent        0.48       existential    26    22.33   
         something quite_unusual       0.47       existential    87    59.75   

                                        LRC  dP2  dP1_simple  dP2_simple  \
polarity l1        l2                                                      
positive all       quite_innovative    4.57 0.00        0.94        0.00   
         something quite_special       4.14 0.00        0.94        0.00   
                   quite_magical       2.39 0.00        0.90        0.00   
                   quite_extraordinary 2.61 0.00        0.83        0.00   
         some      quite_graphic       2.00 0.00        0.66        0.00   
         something quite_profound      1.49 0.00        0.79        0.00   
         some      quite_sophisticated 2.02 0.00        0.65        0.00   
                   quite_elaborate     2.25 0.00        0.61        0.00   
         something quite_unique        2.29 0.00        0.74        0.00   
         some      quite_obscure       1.66 0.00        0.60        0.00   
         something quite_remarkable    1.99 0.00        0.73        0.00   
                   quite_different     2.89 0.01        0.72        0.01   
         often     quite_poor          2.15 0.00        0.56        0.00   
         some      quite_recent        1.47 0.00        0.55        0.00   
         something quite_unusual       1.44 0.00        0.69        0.00   

                                             G2        N      f1    f2  exp_f  
polarity l1        l2                                                          
positive all       quite_innovative    1,153.27  1472077  249279   392  66.38  
         something quite_special       1,048.97  1472077  318343   431  93.21  
                   quite_magical         229.80  1472077  318343   105  22.71  
                   quite_extraordinary   340.35  1472077  318343   197  42.60  
         some      quite_graphic         100.43  1472077  114929    47   3.67  
         something quite_profound        115.93  1472077  318343    77  16.65  
         some      quite_sophisticated   102.72  1472077  114929    49   3.83  
                   quite_elaborate       135.14  1472077  114929    74   5.78  
         something quite_unique          376.89  1472077  318343   303  65.53  
         some      quite_obscure          80.07  1472077  114929    45   3.51  
         something quite_remarkable      240.19  1472077  318343   200  43.25  
                   quite_different     3,637.99  1472077  318343  3156 682.50  
         often     quite_poor            110.99  1472077   92233    63   3.95  
         some      quite_recent           71.40  1472077  114929    47   3.67  
         something quite_unusual         129.55  1472077  318343   126  27.25

In [ ]:
show_metric_top(sig_trig_abbr.filter(like='absolutely_', axis=0), 'dP1', 15)

dP1        quant    f  unexp_f  LRC  \
polarity l1         l2                                                          
positive everything absolutely_ideal     0.77    universal   16    15.46 2.62   
         something  absolutely_new       0.63  existential   83    61.81 2.03   
         everything absolutely_perfect   0.63    universal  120   115.06 4.82   
                    absolutely_true      0.43    universal   73    68.68 3.62   
                    absolutely_normal    0.37    universal   12    11.19 1.55   
                    absolutely_fine      0.36    universal   23    21.42 2.35   
                    absolutely_delicious 0.30    universal   44    40.43 2.66   
         both       absolutely_thrilled  0.26    universal   16    14.13 1.12   
         everyone   absolutely_delighted 0.25    universal   17    15.62 1.72   
         everything absolutely_free      0.24    universal   84    75.65 2.70   
                    absolutely_beautiful 0.23    universal   28    25.02 1.83   
         both       absolutely_right     0.20    universal   20    17.04 1.01   
         everything absolutely_right     0.18    universal   18    15.66 1.08   
         both       absolutely_delicious 0.17    universal   27    22.48 1.10   

                                          dP2  dP1_simple  dP2_simple     G2  \
polarity l1         l2                                                         
positive everything absolutely_ideal     0.00        0.80        0.00  95.97   
         something  absolutely_new       0.00        0.85        0.00 177.64   
         everything absolutely_perfect   0.00        0.65        0.00 634.32   
                    absolutely_true      0.00        0.45        0.00 311.31   
                    absolutely_normal    0.00        0.40        0.00  47.43   
                    absolutely_fine      0.00        0.39        0.00  89.48   
                    absolutely_delicious 0.00        0.33        0.00 154.40   
         both       absolutely_thrilled  0.00        0.29        0.00  44.61   
         everyone   absolutely_delighted 0.00        0.27        0.00  58.41   
         everything absolutely_free      0.00        0.27        0.00 257.43   
                    absolutely_beautiful 0.00        0.25        0.00  81.74   
         both       absolutely_right     0.00        0.23        0.00  46.12   
         everything absolutely_right     0.00        0.21        0.00  45.29   
         both       absolutely_delicious 0.00        0.20        0.00  55.77   

                                                N      f1   f2  exp_f  
polarity l1         l2                                                 
positive everything absolutely_ideal      1472077   39516   20   0.54  
         something  absolutely_new        1472077  318343   98  21.19  
         everything absolutely_perfect    1472077   39516  184   4.94  
                    absolutely_true       1472077   39516  161   4.32  
                    absolutely_normal     1472077   39516   30   0.81  
                    absolutely_fine       1472077   39516   59   1.58  
                    absolutely_delicious  1472077   39516  133   3.57  
         both       absolutely_thrilled   1472077   50007   55   1.87  
         everyone   absolutely_delighted  1472077   32327   63   1.38  
         everything absolutely_free       1472077   39516  311   8.35  
                    absolutely_beautiful  1472077   39516  111   2.98  
         both       absolutely_right      1472077   50007   87   2.96  
         everything absolutely_right      1472077   39516   87   2.34  
         both       absolutely_delicious  1472077   50007  133   4.52

In [ ]:
show_metric_top(sig_trig_abbr.filter(like='downright_', axis=0), 'dP1', 15)

dP1        quant   f  unexp_f  LRC  \
polarity l1        l2                                                          
positive sometimes downright_frustrating 0.73  existential  14    13.09 1.40   
                   downright_difficult   0.71  existential  16    14.93 1.72   
         or        downright_impossible  0.47  existential  57    38.81 1.04   
         some      downright_awful       0.36  existential  34    27.91 1.33   
                   downright_ugly        0.36  existential  33    27.07 1.29   
         sometimes downright_painful     0.34  existential  24    20.90 1.46   
         some      downright_hilarious   0.31  existential  33    26.36 1.09   
         sometimes downright_scary       0.31  existential  40    34.32 1.77   
                   downright_nasty       0.27  existential  22    18.50 1.02   
         some      downright_dangerous   0.25  existential  51    38.82 1.13   
         sometimes downright_bizarre     0.23  existential  30    24.67 1.14   

                                          dP2  dP1_simple  dP2_simple    G2  \
polarity l1        l2                                                         
positive sometimes downright_frustrating 0.00        0.78        0.00 64.81   
                   downright_difficult   0.00        0.76        0.00 72.86   
         or        downright_impossible  0.00        0.70        0.00 83.34   
         some      downright_awful       0.00        0.44        0.00 73.72   
                   downright_ugly        0.00        0.43        0.00 71.27   
         sometimes downright_painful     0.00        0.39        0.00 65.17   
         some      downright_hilarious   0.00        0.39        0.00 63.22   
         sometimes downright_scary       0.00        0.36        0.00 99.99   
                   downright_nasty       0.00        0.32        0.00 49.67   
         some      downright_dangerous   0.00        0.33        0.00 80.02   
         sometimes downright_bizarre     0.00        0.29        0.00 61.04   

                                                N      f1   f2  exp_f  
polarity l1        l2                                                  
positive sometimes downright_frustrating  1472077   74707   18   0.91  
                   downright_difficult    1472077   74707   21   1.07  
         or        downright_impossible   1472077  326614   82  18.19  
         some      downright_awful        1472077  114929   78   6.09  
                   downright_ugly         1472077  114929   76   5.93  
         sometimes downright_painful      1472077   74707   61   3.10  
         some      downright_hilarious    1472077  114929   85   6.64  
         sometimes downright_scary        1472077   74707  112   5.68  
                   downright_nasty        1472077   74707   69   3.50  
         some      downright_dangerous    1472077  114929  156  12.18  
         sometimes downright_bizarre      1472077   74707  105   5.33

In [ ]:
show_metric_top(sig_trig_abbr.filter(like='definitely_', axis=0), 'dP1', 15)

dP1        quant    f  unexp_f  LRC  \
polarity l1        l2                                                       
positive something definitely_wrong  0.77  existential  359   280.28 4.35   
         some      definitely_better 0.57  existential   43    37.85 2.36   

                                      dP2  dP1_simple  dP2_simple       G2  \
polarity l1        l2                                                        
positive something definitely_wrong  0.00        0.99        0.00 1,049.42   
         some      definitely_better 0.00        0.65        0.00   137.72   

                                            N      f1   f2  exp_f  
polarity l1        l2                                              
positive something definitely_wrong   1472077  318343  364  78.72  
         some      definitely_better  1472077  114929   66   5.15

In [ ]:
show_metric_top(sig_trig_abbr.filter(like='utterly_', axis=0), 'dP1', 15)

,,,dP1,quant,f,unexp_f,LRC,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f
polarity,l1,l2,,,,,,,,,,,,,
positive,something,utterly_different,0.59,existential,63,46.13,1.58,0.00,0.81,0.00,123.89,1472077,318343,78,16.87


In [ ]:
show_metric_top(sig_trig_abbr.filter(like='that_', axis=0), 'dP1', 15)

dP1        quant   f  unexp_f  LRC  dP2  \
polarity l1        l2                                                        
positive all       that_different 0.74    universal  42    34.21 1.41 0.00   
                   that_important 0.60    universal  41    32.03 1.39 0.00   
                   that_great     0.59    universal  44    34.18 1.44 0.00   
         sometimes that_simple    0.41  existential  26    23.16 1.83 0.00   
         always    that_easy      0.32    universal  22    18.76 1.16 0.00   

                                   dP1_simple  dP2_simple     G2        N  \
polarity l1        l2                                                       
positive all       that_different        0.91        0.00 123.48  1472077   
                   that_important        0.77        0.00  93.38  1472077   
                   that_great            0.76        0.00  97.37  1472077   
         sometimes that_simple           0.46        0.00  80.79  1472077   
         always    that_easy             0.38        0.00  54.10  1472077   

                                       f1  f2  exp_f  
polarity l1        l2                                 
positive all       that_different  249279  46   7.79  
                   that_important  249279  53   8.97  
                   that_great      249279  58   9.82  
         sometimes that_simple      74707  56   2.84  
         always    that_easy        82200  58   3.24

In [ ]:
show_metric_top(sig_trig_abbr.filter(like='particularly_', axis=0), 'dP1', 15)

dP1             quant   f  \
polarity l1        l2                                                   
positive something particularly_special    0.67       existential  79   
         always    particularly_fond       0.54         universal  28   
         many      particularly_concerned  0.37  hedged_universal  19   
         always    particularly_interested 0.32         universal  32   

                                            unexp_f  LRC  dP2  dP1_simple  \
polarity l1        l2                                                       
positive something particularly_special       59.75 2.10 0.00        0.89   
         always    particularly_fond          25.38 2.21 0.00        0.60   
         many      particularly_concerned     17.50 1.93 0.00        0.40   
         always    particularly_interested    27.20 1.50 0.00        0.37   

                                            dP2_simple     G2        N  \
polarity l1        l2                                                    
positive something particularly_special           0.00 184.28  1472077   
         always    particularly_fond              0.00 100.35  1472077   
         many      particularly_concerned         0.00  69.18  1472077   
         always    particularly_interested        0.00  77.34  1472077   

                                                f1  f2  exp_f  
polarity l1        l2                                          
positive something particularly_special     318343  89  19.25  
         always    particularly_fond         82200  47   2.62  
         many      particularly_concerned    47133  47   1.50  
         always    particularly_interested   82200  86   4.80

In [ ]:
show_metric_top(sig_trig_abbr.filter(like='increasingly_', axis=0), 'dP1', 15)

dP1             quant   f  unexp_f  LRC  \
polarity l1   l2                                                                
positive many increasingly_concerned 0.72  hedged_universal  18    17.23 2.65   
              increasingly_worried   0.56  hedged_universal  10     9.46 1.41   

                                      dP2  dP1_simple  dP2_simple    G2  \
polarity l1   l2                                                          
positive many increasingly_concerned 0.00        0.75        0.00 97.30   
              increasingly_worried   0.00        0.59        0.00 46.25   

                                            N     f1  f2  exp_f  
polarity l1   l2                                                 
positive many increasingly_concerned  1472077  47133  24   0.77  
              increasingly_worried    1472077  47133  17   0.54

In [ ]:
show_metric_top(sig_trig_abbr.filter(like='quite_', axis=0), 'dP1', 15)

dP1             quant     f  unexp_f  \
polarity l1        l2                                                          
positive all       quite_innovative    0.77         universal   370   303.62   
         something quite_special       0.72       existential   404   310.79   
                   quite_magical       0.69       existential    95    72.29   
                   quite_extraordinary 0.62       existential   164   121.40   
         some      quite_graphic       0.58       existential    31    27.33   
         something quite_profound      0.58       existential    61    44.35   
         some      quite_sophisticated 0.58       existential    32    28.17   
                   quite_elaborate     0.53       existential    45    39.22   
         something quite_unique        0.52       existential   224   158.47   
         some      quite_obscure       0.52       existential    27    23.49   
         something quite_remarkable    0.51       existential   146   102.75   
                   quite_different     0.50       existential  2268 1,585.50   
         often     quite_poor          0.49  hedged_universal    35    31.05   
         some      quite_recent        0.48       existential    26    22.33   
         something quite_unusual       0.47       existential    87    59.75   

                                        LRC  dP2  dP1_simple  dP2_simple  \
polarity l1        l2                                                      
positive all       quite_innovative    4.57 0.00        0.94        0.00   
         something quite_special       4.14 0.00        0.94        0.00   
                   quite_magical       2.39 0.00        0.90        0.00   
                   quite_extraordinary 2.61 0.00        0.83        0.00   
         some      quite_graphic       2.00 0.00        0.66        0.00   
         something quite_profound      1.49 0.00        0.79        0.00   
         some      quite_sophisticated 2.02 0.00        0.65        0.00   
                   quite_elaborate     2.25 0.00        0.61        0.00   
         something quite_unique        2.29 0.00        0.74        0.00   
         some      quite_obscure       1.66 0.00        0.60        0.00   
         something quite_remarkable    1.99 0.00        0.73        0.00   
                   quite_different     2.89 0.01        0.72        0.01   
         often     quite_poor          2.15 0.00        0.56        0.00   
         some      quite_recent        1.47 0.00        0.55        0.00   
         something quite_unusual       1.44 0.00        0.69        0.00   

                                             G2        N      f1    f2  exp_f  
polarity l1        l2                                                          
positive all       quite_innovative    1,153.27  1472077  249279   392  66.38  
         something quite_special       1,048.97  1472077  318343   431  93.21  
                   quite_magical         229.80  1472077  318343   105  22.71  
                   quite_extraordinary   340.35  1472077  318343   197  42.60  
         some      quite_graphic         100.43  1472077  114929    47   3.67  
         something quite_profound        115.93  1472077  318343    77  16.65  
         some      quite_sophisticated   102.72  1472077  114929    49   3.83  
                   quite_elaborate       135.14  1472077  114929    74   5.78  
         something quite_unique          376.89  1472077  318343   303  65.53  
         some      quite_obscure          80.07  1472077  114929    45   3.51  
         something quite_remarkable      240.19  1472077  318343   200  43.25  
                   quite_different     3,637.99  1472077  318343  3156 682.50  
         often     quite_poor            110.99  1472077   92233    63   3.95  
         some      quite_recent           71.40  1472077  114929    47   3.67  
         something quite_unusual         129.55  1472077  318343   126  27.25